In [135]:
import cv2
import numpy as np
import pandas as pd
import itertools

In [136]:
##### tarefas para fazer
## Algoritmos de detecção
# -- harris - OK
# -- sift - OK
# -- star/brisk (pendente)
# -- entender parâmetros harris, sift, star e brisk
# -- criar experimento para 25, 50, 100, 500, 1000 features
########################
## Algoritmos de descrição
# -- restringir área de checagem de pontos
# -- criar algoritmo para contar falsos positivos
## Imagem de teste
# -- preciso escolher qual o terceiro par de imagens para testar

# Experimento com detectores

# Códigos detectores

In [137]:
def detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.04):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gray_image_f32 = np.float32(gray_image)

    # Applying the function 
    dst = cv2.cornerHarris(gray_image_f32, blockSize, ksize, k) 
  
    # dilate to mark the corners 
    dst = cv2.dilate(dst, None)
    
    ret, dst = cv2.threshold(dst,threshold*dst.max(),255,0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]

    # draw keypoints
    # image[dst > threshold * dst.max()] = [0, 255, 0]
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(255, 0, 0), flags=0)

    return keypoints, kp_image


In [138]:
def detectSIFTKeypoints(image, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, enable_precise_upscale=False):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma, enable_precise_upscale) 
    kp, des = sift.detectAndCompute(gray_image, None) 
    
    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 255, 0), flags=0) 

    return kp, kp_image 

In [139]:
def detectStarKeypoints(image, max_size = 41, response_threshold = 30, line_threshold_projected = 10,
                        line_threshold_binarized = 8, suppress_nonmax_size = 5):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    star = cv2.xfeatures2d.StarDetector_create(maxSize= max_size, 
                                        responseThreshold = response_threshold,
                                        lineThresholdProjected = line_threshold_projected,
                                        lineThresholdBinarized = line_threshold_binarized,
                                        suppressNonmaxSize = suppress_nonmax_size)
    kp = star.detect(gray_image, None)    

    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 0, 255), flags=0) 

    return kp, kp_image 

In [142]:
# image = cv2.imread('../data/imgs/dsc07631.jpg')
# kp, kp_image = detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.02)
# print(len(kp))
# cv2.imshow('Star', kp_image) 
# cv2.waitKey() 

In [6]:
# def detectORBKeypoints(image, nfeatures=500, scaleFactor = 1.2, nlevels=8, edgeThreshold=31, WTA_K=2, patchSize=31):
#     # Reading the image and converting the image to B/W 
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
#     # Applying the function 
#     orb = cv2.ORB_create(
#         nfeatures=nfeatures,
#         scaleFactor=scaleFactor,
#         nlevels=nlevels, 
#         edgeThreshold=edgeThreshold, 
#         WTA_K=WTA_K, 
#         patchSize=patchSize, 
#     ) 
#     kp, des = orb.detectAndCompute(gray_image, None)    

#     # Applying the function 
#     kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 0, 255), flags=0) 

#     return kp, kp_image 

## Código comparando pontos

In [172]:
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min

def computeDistacesKeypoints(pts1, pts2, threshold=1):
    array_pts1 = np.asarray(pts1)
    array_pts2 = np.asarray(pts2)

    if array_pts1.shape[0] > 0 and array_pts2.shape[0] > 0:
        dists = pairwise_distances_argmin_min(array_pts1, array_pts2)  
        matches = [pts2[pt] for pt, dist in zip(dists[0], dists[1]) if dist <= threshold]
    else:
        matches = []
        
    return matches

In [167]:
def extract_and_save_features_csv(params, algol, path):
    if algol == 'star':
        func_var = detectStarKeypoints
    elif algol == 'harris':
        func_var = detectHarrisKeypoints
    else:
        func_var = detectSIFTKeypoints
    
    cols_name = [algol + '_' + param for param in list(params.keys())[1:]]
    cols_result = [algol + '_' + 'qtd_keypoints', algol + '_' + 'keypoints']
    df = pd.DataFrame(columns=cols_name+cols_result)
    
    keys = list(params)
    for values in itertools.product(*map(params.get, keys)):
        kps, kp_image = func_var(**dict(zip(keys, values)))
        kps_ = [(kp.pt[0], kp.pt[1]) for kp in kps] #kp.angle, kp.response, kp.octave, kp.class_id) for kp in kps]

        df.loc[len(df)] = list(values[1:]) + [len(kps_), kps_]

    df.to_csv(path, index=False)
    return df


def convert_to_kp(string_points):
    string_converted = list(eval(string_points))
    # kps = [cv2.KeyPoint(x[0], x[1], 13) for x in string_converted]
    # for p in string_converted:
    #     kp = cv2.KeyPoint(x=float(p[0]), y=float(p[1]), size=float(p[2]), angle=float(p[3]),
    #                 response=float(p[4]), octave=int(p[5]), class_id=int(p[6]))
    #     kps.append(kp)
    
    return string_converted
        

def compare_and_save_match_points(df1, df2, kp1_col, kp2_col, path):
    cols1 = list(df1.columns[:-1])
    cols2 = list(df2.columns[:-1])
    extra_cols = ['qtd_matches', 'match_keypoints']
    
    df = pd.DataFrame(columns=cols1+cols2+extra_cols)
    
    for idx1 in df1.index:
        kp1 = df1.loc[idx1][kp1_col]
        print(f'passou {idx1}')
        for idx2 in df2.index:
            kp2 = df2.loc[idx2][kp2_col]
            matches = computeDistacesKeypoints(kp1, kp2, threshold=2)

            df.loc[len(df)] = df1.loc[idx1][cols1].tolist() + df2.loc[idx2][cols2].tolist() + [len(matches), matches]
    
    df.to_csv(path, index=False)

    return df

## Salvando combinações de parâmetros dos algoritmos

In [ ]:
image = cv2.imread('../data/imgs/dsc07631.jpg')

params_harris = {
    'image': [image],
    'threshold' : [0.005, 0.01, 0.05, 0.08],
    'blockSize' : [2, 4, 6, 8],
    'ksize' : [3, 5, 7, 9],
    'k' : [0.02, 0.04, 0.08, 0.16]
}

params_sift = {
    'image': [image],
    # 'nfeatures': [50, 100, 500, 1000],
    'nOctaveLayers' : [3, 5, 7, 9],
    'contrastThreshold' : [0.02, 0.04, 0.08, 0.1],
    'edgeThreshold' : [5, 10, 20, 40],
    'sigma' : [0.8, 1.6, 3.2, 6.4]
}

params_star = {
    'image': [image],
    'max_size': [11, 21, 41, 81],
    'response_threshold': [5, 10, 20, 30],
    'line_threshold_projected': [5, 10, 20, 30],
    'line_threshold_binarized': [4, 8, 16, 32],
    'suppress_nonmax_size': [2, 3, 5, 7]
}

df_harris = extract_and_save_features_csv(params_harris, 'harris', '../data/results/dsc07631/harris_keypoints_1.csv')
df_sift = extract_and_save_features_csv(params_sift, 'sift', '../data/results/dsc07631/sift_keypoints_1.csv')
df_orb = extract_and_save_features_csv(params_star, 'star', '../data/results/dsc07631/star_keypoints_1.csv')


In [158]:
df_harris = pd.read_csv('../data/results/dsc07631/harris_keypoints_1.csv')
df_sift = pd.read_csv('../data/results/dsc07631/sift_keypoints_1.csv')
df_star = pd.read_csv('../data/results/dsc07631/star_keypoints_1.csv')

In [159]:
kps1 = []
for idx in df_harris.index.tolist():
    kps1.append(convert_to_kp(df_harris['harris_keypoints'].loc[idx]))


In [160]:
kps2 = []
for idx in df_sift.index.tolist():
    kps2.append(convert_to_kp(df_sift['sift_keypoints'].loc[idx]))

In [161]:
kps3 = []
for idx in df_star.index.tolist():
    kps3.append(convert_to_kp(df_star['star_keypoints'].loc[idx]))

In [162]:
kps1

[[(323.1404724121094, 238.43824768066406),
  (11.443663597106934, 6.556921482086182),
  (34.00852966308594, 1.7629450559616089),
  (73.59446716308594, 1.690458059310913),
  (77.03231048583984, 3.402292490005493),
  (93.74288940429688, 0.881417989730835),
  (515.927490234375, 2.6884162425994873),
  (587.8070068359375, 0.5245643854141235),
  (615.255615234375, 2.762990713119507),
  (633.4849853515625, 0.1806456446647644),
  (530.4000244140625, 3.5999999046325684),
  (541.8428955078125, 12.545638084411621),
  (11.443926811218262, 6.558378219604492),
  (28.612895965576172, 16.915733337402344),
  (519.6922607421875, 19.752685546875),
  (531.0, 17.0),
  (8.804922103881836, 21.87242889404297),
  (146.69125366210938, 23.752553939819336),
  (632.806640625, 22.023143768310547),
  (71.46634674072266, 25.70682144165039),
  (614.1219482421875, 27.684904098510742),
  (133.3186798095703, 29.596555709838867),
  (208.53048706054688, 28.96262550354004),
  (7.5, 33.0),
  (120.6021499633789, 34.0859527587

In [163]:
df_harris['harris_keypoints'] = kps1
df_sift['sift_keypoints'] = kps2
df_star['star_keypoints'] = kps3

In [164]:
df_hs = compare_and_save_match_points(df_harris, df_sift, 'harris_keypoints', 'sift_keypoints', '../data/results/dsc07631/harris_sift_matches.csv')

passou 0
passou 1
passou 2
passou 3
passou 4
passou 5
passou 6
passou 7
passou 8
passou 9
passou 10
passou 11
passou 12
passou 13
passou 14
passou 15
passou 16
passou 17
passou 18
passou 19
passou 20
passou 21
passou 22
passou 23
passou 24
passou 25
passou 26
passou 27
passou 28
passou 29
passou 30
passou 31
passou 32
passou 33
passou 34
passou 35
passou 36
passou 37
passou 38
passou 39
passou 40
passou 41
passou 42
passou 43
passou 44
passou 45
passou 46
passou 47
passou 48
passou 49
passou 50
passou 51
passou 52
passou 53
passou 54
passou 55
passou 56
passou 57
passou 58
passou 59
passou 60
passou 61
passou 62
passou 63
passou 64
passou 65
passou 66
passou 67
passou 68
passou 69
passou 70
passou 71
passou 72
passou 73
passou 74
passou 75
passou 76
passou 77
passou 78
passou 79
passou 80
passou 81
passou 82
passou 83
passou 84
passou 85
passou 86
passou 87
passou 88
passou 89
passou 90
passou 91
passou 92
passou 93
passou 94
passou 95
passou 96
passou 97
passou 98
passou 99
passou 100

In [173]:
df_hst = compare_and_save_match_points(df_harris, df_star, 'harris_keypoints', 'star_keypoints', '../data/results/dsc07631/harris_star_matches.csv')

passou 0
passou 1
passou 2
passou 3
passou 4
passou 5
passou 6
passou 7
passou 8
passou 9
passou 10
passou 11
passou 12
passou 13
passou 14
passou 15
passou 16
passou 17
passou 18
passou 19
passou 20
passou 21
passou 22
passou 23
passou 24
passou 25
passou 26
passou 27
passou 28
passou 29
passou 30
passou 31
passou 32
passou 33
passou 34
passou 35
passou 36
passou 37
passou 38
passou 39
passou 40
passou 41
passou 42
passou 43
passou 44
passou 45
passou 46
passou 47
passou 48
passou 49
passou 50
passou 51
passou 52
passou 53
passou 54
passou 55
passou 56
passou 57
passou 58


KeyboardInterrupt: 

In [ ]:
df_ss = compare_and_save_match_points(df_sift, df_star, 'sift_keypoints', 'star_keypoints', '../data/results/dsc07631/sift_star_matches.csv')

passou 0
passou 1
passou 2
passou 3
passou 4
passou 5
passou 6
passou 7
passou 8
passou 9
passou 10
passou 11
passou 12
passou 13
passou 14
passou 15
passou 16
passou 17
passou 18
passou 19
passou 20
passou 21
passou 22
passou 23
passou 24
passou 25
passou 26
passou 27
passou 28
passou 29
passou 30
passou 31
passou 32
passou 33
passou 34
passou 35
passou 36
passou 37
passou 38
passou 39
passou 40
passou 41
passou 42
passou 43
passou 44
passou 45
passou 46
passou 47
passou 48
passou 49
passou 50
passou 51
passou 52
passou 53
passou 54
passou 55
passou 56
passou 57
passou 58
passou 59
passou 60
passou 61
passou 62
passou 63
passou 64
passou 65
passou 66
passou 67
passou 68
passou 69
passou 70
passou 71
passou 72
passou 73
passou 74
passou 75
passou 76
passou 77
passou 78
passou 79
passou 80


## Segundo Experimento

In [389]:
df_harris.head()

,harris_threshold,harris_blockSize,harris_ksize,harris_k,harris_qtd_keypoints,harris_keypoints
0,0.01,2,3,0.02,10493,"[(71.0, 0.0), (72.0, 0.0), (73.0, 0.0), (74.0,..."
1,0.01,2,3,0.04,9288,"[(71.0, 0.0), (72.0, 0.0), (73.0, 0.0), (74.0,..."
2,0.01,2,3,0.16,5947,"[(98.0, 2.0), (99.0, 2.0), (100.0, 2.0), (98.0..."
3,0.01,2,5,0.02,8844,"[(71.0, 0.0), (72.0, 0.0), (73.0, 0.0), (74.0,..."
4,0.01,2,5,0.04,7534,"[(71.0, 0.0), (72.0, 0.0), (73.0, 0.0), (74.0,..."


In [ ]:
matches_hs = computeDistacesKeypoints(h_kp, s_kp, threshold=2)
matches_ho = computeDistacesKeypoints(h_kp, o_kp, threshold=2)
matches_so = computeDistacesKeypoints(s_kp, o_kp, threshold=2)

## Primeiro experimento
- Rodando com os parâmetros default dos algoritmos

In [271]:
image = cv2.imread('../data/imgs/dsc07631.jpg')
h_kp, harris_image = detectHarrisKeypoints(image)
s_kp, sift_image = detectSIFTKeypoints(image)
o_kp, orb_image = detectStarKeypoints(image)


### threshold para matches de detectores 1

In [272]:
matches_hs = computeDistacesKeypoints(h_kp, s_kp, threshold=1)
matches_ho = computeDistacesKeypoints(h_kp, o_kp, threshold=1)
matches_so = computeDistacesKeypoints(s_kp, o_kp, threshold=1)

In [258]:
print('----- Resultados -----')
print(f'QTD Harris Keypoints:   {len(h_kp)}')
print(f'QTD SIFT Keypoints:   {len(s_kp)}')
print(f'QTD ORB Keypoints:   {len(o_kp)}')
print('----- Matches -----')
print(f'Matches Keypoints Harris/SIFT:   {len(matches_hs)}')
print(f'Matches Keypoints Harris/ORB:   {len(matches_ho)}')
print(f'Matches Keypoints SIFT/ORB:   {len(matches_so)}')

----- Resultados -----
QTD Harris Keypoints:   9288
QTD SIFT Keypoints:   1518
QTD ORB Keypoints:   500
----- Matches -----
Matches Keypoints Harris/SIFT:   645
Matches Keypoints Harris/ORB:   666
Matches Keypoints SIFT/ORB:   80


In [259]:
kp_matches_hs = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_hs]
kp_matches_ho = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_ho]
kp_matches_so = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_so]

In [260]:
kp1_image = cv2.drawKeypoints(image, kp_matches_hs, None, color=(0, 0, 255), flags=0)
kp2_image = cv2.drawKeypoints(image, kp_matches_ho, None, color=(0, 255, 0), flags=0)
kp3_image = cv2.drawKeypoints(image, kp_matches_so, None, color=(255, 0, 0), flags=0)

In [262]:
# cv2.imshow('ORB', kp_matches_hs)

### threshold para matches de detectores 2

In [263]:
matches_hs = computeDistacesKeypoints(h_kp, s_kp, threshold=2)
matches_ho = computeDistacesKeypoints(h_kp, o_kp, threshold=2)
matches_so = computeDistacesKeypoints(s_kp, o_kp, threshold=2)

In [264]:
print('----- Resultados -----')
print(f'QTD Harris Keypoints:   {len(h_kp)}')
print(f'QTD SIFT Keypoints:   {len(s_kp)}')
print(f'QTD ORB Keypoints:   {len(o_kp)}')
print('----- Matches -----')
print(f'Matches Keypoints Harris/SIFT:   {len(matches_hs)}')
print(f'Matches Keypoints Harris/ORB:   {len(matches_ho)}')
print(f'Matches Keypoints SIFT/ORB:   {len(matches_so)}')

----- Resultados -----
QTD Harris Keypoints:   9288
QTD SIFT Keypoints:   1518
QTD ORB Keypoints:   500
----- Matches -----
Matches Keypoints Harris/SIFT:   2342
Matches Keypoints Harris/ORB:   1430
Matches Keypoints SIFT/ORB:   125


In [265]:
kp_matches_hs = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_hs]
kp_matches_ho = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_ho]
kp_matches_so = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_so]

In [266]:
kp1_image = cv2.drawKeypoints(image, kp_matches_hs, None, color=(0, 0, 255), flags=0)
kp2_image = cv2.drawKeypoints(image, kp_matches_ho, None, color=(0, 255, 0), flags=0)
kp3_image = cv2.drawKeypoints(image, kp_matches_so, None, color=(255, 0, 0), flags=0)

In [268]:
# cv2.imshow('Harris/SIFT', kp1_image)
# cv2.waitKey() 

# SIFT Descriptor

In [9]:
def calculate_slope(pt1, pt2):
    slope = (pt2[1] - pt1[1]) / (pt2[0] - pt1[0])
    return abs(slope)


def verify_slope(match, trainKeypoints, queryKeypoints, q1, q2):
    slope = calculate_slope(trainKeypoints[match.trainIdx].pt, queryKeypoints[match.queryIdx].pt) 
    if slope <= q2 and slope > q1:
        return True
    else:
        return False


def filter_matches_by_slope(matches, matchesMask, trainKeypoints, queryKeypoints, q1, q2):
    slope_list = [calculate_slope(trainKeypoints[matches[i][0].trainIdx].pt, queryKeypoints[matches[i][0].queryIdx].pt) for i, m in enumerate(matchesMask) if m[0] > 0]
    q1_, q2_ = np.percentile(slope_list, [q1 ,q2])
    matchesMask_ = [ [1,0] if m[0] > 0 and verify_slope(matches[i][0], trainKeypoints, queryKeypoints, q1_, q2_) else [0,0] for i, m in enumerate(matchesMask)]
    
    qtd_fp = len([m[0] for m, m_ in zip(matchesMask, matchesMask_) if m[0] != m_[0]])
    qtd_matches = len([m[0] for m in matchesMask_ if m[0] > 0 ])
    
    mismatch_train_keypoints = len(trainKeypoints) - qtd_matches
    mismatch_query_keypoints = len(queryKeypoints) - qtd_matches

    return matchesMask_, qtd_fp, qtd_matches, mismatch_train_keypoints, mismatch_query_keypoints


In [22]:
def matchDetectedKeypoints(train_img, query_img, ratio_test=0.7, filter_slope=(2,98), detector='harris', descriptor='orb', dect_kargs={}, des_kargs={}):
    query_img_bw = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY) 
    train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)

    if descriptor == 'orb':
        des = cv2.ORB.create(**des_kargs)
    else:
        des = cv2.SIFT.create(**des_kargs)
    
    if detector == 'orb':
        func_var = detectStarKeypoints
    elif detector == 'harris':
        func_var = detectHarrisKeypoints
    else:
        func_var = detectSIFTKeypoints

    queryKeypoints, _ = func_var(query_img, **dect_kargs)
    trainKeypoints, _ = func_var(train_img, **dect_kargs)
    
    _,queryDescriptors = des.compute(query_img_bw, queryKeypoints) 
    _,trainDescriptors = des.compute(train_img_bw, trainKeypoints)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=100)   # or pass empty dictionary

    # flann = cv2.FlannBasedMatcher(index_params, search_params)
    # matches = flann.knnMatch(queryDescriptors, trainDescriptors, k=2)

    matcher = cv2.BFMatcher() 
    matches = matcher.knnMatch(queryDescriptors,trainDescriptors, k=2) 

    # Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]

    for i,(m,n) in enumerate(matches):
        if m.distance < ratio_test*n.distance:
            matchesMask[i]=[1,0]

    m = filter_matches_by_slope(matches, matchesMask, trainKeypoints, queryKeypoints, filter_slope[0], filter_slope[1])
    matchesMask_, qtd_fp, qtd_matches, mismatch_train_keypoints, mismatch_query_keypoints = m 

    print('---- Resultado ---- ')
    print(f'Quantidade de matches: {qtd_matches}')
    print(f'Quantidade de falso positivos: {qtd_fp}')
    print(f'Quantidade de pontos sem match (train_image): {mismatch_train_keypoints}')
    print(f'Quantidade de pontos sem match (query_image): {mismatch_query_keypoints}')

    draw_params = dict(matchColor = (0,255,0),
                    singlePointColor = (255,0,0),
                    matchesMask = matchesMask_,
                    flags = cv2.DrawMatchesFlags_DEFAULT)
    final_img = cv2.drawMatchesKnn(query_img,queryKeypoints,train_img,trainKeypoints,matches,None,**draw_params)

    
    final_img = cv2.resize(final_img, (1280,960))

    return final_img 

    

In [23]:
query_img = cv2.imread('../data/imgs_teste/IMG_20231129_083404.jpg') 
train_img = cv2.imread('../data/imgs_teste/IMG_20231129_083413.jpg') 
query_img_ = cv2.resize(query_img, (640,480))
train_img_ = cv2.resize(train_img, (640,480))

In [33]:
detect_sift = {
    'nfeatures':100,
    'nOctaveLayers':8, 
    # 'contrastThreshold':0.04, 
    'edgeThreshold':31, 
    'sigma':1.2, 
    'enable_precise_upscale':True
}

In [34]:
final_img = matchDetectedKeypoints(train_img_, query_img_, 1, filter_slope=(0,75), detector='sift', descriptor='orb', dect_kargs=detect_sift, des_kargs={})

(< cv2.KeyPoint 000001E81051CAB0>, < cv2.KeyPoint 000001E81071CD80>, < cv2.KeyPoint 000001E81071CB70>, < cv2.KeyPoint 000001E81071C960>, < cv2.KeyPoint 000001E81071CCF0>, < cv2.KeyPoint 000001E81071C8D0>, < cv2.KeyPoint 000001E81071CC30>, < cv2.KeyPoint 000001E81071CDE0>, < cv2.KeyPoint 000001E81071C840>, < cv2.KeyPoint 000001E81071CA20>, < cv2.KeyPoint 000001E81071CAE0>, < cv2.KeyPoint 000001E81071CC60>, < cv2.KeyPoint 000001E81071C870>, < cv2.KeyPoint 000001E81071CBD0>, < cv2.KeyPoint 000001E81071C7B0>, < cv2.KeyPoint 000001E81071C8A0>, < cv2.KeyPoint 000001E81071C9C0>, < cv2.KeyPoint 000001E81071C750>, < cv2.KeyPoint 000001E81071CC00>, < cv2.KeyPoint 000001E81071C930>, < cv2.KeyPoint 000001E81071CFC0>, < cv2.KeyPoint 000001E81071CED0>, < cv2.KeyPoint 000001E81071C9F0>, < cv2.KeyPoint 000001E81071CC90>, < cv2.KeyPoint 000001E81071CF90>, < cv2.KeyPoint 000001E80FE9FCC0>, < cv2.KeyPoint 000001E80FE9FC30>, < cv2.KeyPoint 000001E8103B59C0>, < cv2.KeyPoint 000001E8103F6D50>, < cv2.KeyPoin

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 68187556800 bytes in function 'cv::OutOfMemoryError'


In [19]:
# Show the final image 
cv2.imshow("Matches", final_img) 
cv2.waitKey()

-1

In [38]:
import cv2 as cv

In [24]:
# Read the query image as query_img 
# and train image This query image 
# is what you need to find in train image 
# Save it in the same directory 
# with the name image.jpg
   
query_img = cv2.imread('../data/imgs_teste/IMG_20231129_083404.jpg') 
train_img = cv2.imread('../data/imgs_teste/IMG_20231129_083413.jpg') 
query_img = cv2.resize(query_img, (640,480))
train_img = cv2.resize(train_img, (640,480))
# Convert it to grayscale 
query_img_bw = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY) 
train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY) 
   
# Initialize the ORB detector algorithm 
orb = cv2.ORB_create(nfeatures=100)
sift = cv2.SIFT_create(nfeatures=100)
brisk = cv2.BRISK_create()

star = cv2.xfeatures2d.StarDetector_create()
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

# Now detect the keypoints and compute 
# the descriptors for the query image 
# and train image 

queryKeypoints = sift.detect(query_img_bw,None) 
trainKeypoints, trainDescriptors = brisk.compute(train_img_bw,queryKeypoints)
  
# Initialize the Matcher for matching 
# the keypoints and then match the 
# keypoints 

# matcher = cv2.BFMatcher() 
# matches = matcher.knnMatch(queryDescriptors,trainDescriptors, k=2) 

# good_matches = []
# for m, n in matches:
#     if m.distance < 0.6 * n.distance:
#         good_matches.append(m)
   
# draw the matches to the final image 
# containing both the images the drawMatches() 
# function takes both images and keypoints 
# and outputs the matched query image with 
# its train image 

# final_img = cv2.drawMatches(query_img, queryKeypoints, train_img, trainKeypoints, good_matches, None, flags=2) 


FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=100)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(queryDescriptors, trainDescriptors,k=2)
# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]

for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        matchesMask[i]=[1,0]

m = filter_matches_by_slope(matches, matchesMask, trainKeypoints, queryKeypoints, 2, 98)
matchesMask_, qtd_fp, qtd_matches, mismatch_train_keypoints, mismatch_query_keypoints = m 

print('---- Resultado ---- ')
print(f'Quantidade de matches: {qtd_matches}')
print(f'Quantidade de falso positivos: {qtd_fp}')
print(f'Quantidade de pontos sem match (train_image): {mismatch_train_keypoints}')
print(f'Quantidade de pontos sem match (query_image): {mismatch_query_keypoints}')

draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask_,
                   flags = cv2.DrawMatchesFlags_DEFAULT)
final_img = cv2.drawMatchesKnn(query_img,queryKeypoints,train_img,trainKeypoints,matches,None,**draw_params)

   
final_img = cv2.resize(final_img, (1280,960)) 
  
# Show the final image 
cv2.imshow("Matches", final_img) 
cv2.waitKey()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\flann\src\miniflann.cpp:336: error: (-210:Unsupported format or combination of formats) in function 'cv::flann::buildIndex_'
> type=0
> 

# ORB Descriptor

In [2]:
import cv2 as cv

In [8]:
cv2.xfeatures2d.SIFT_create()

< cv2.SIFT 0000028B6AF50FB0>